# Objective

1. Repositioning strategies



In [1]:
import pandas as pd

import numpy as np

import networkx as nx

import random

from shapely.geometry import Point, Polygon


import geopandas as gp

In [2]:
'''Param'''

s_sec=25200

e_sec=36000

Start_step=2520

End_step=3600

Max_waiting=12

radius=2000

speed=10 # 10 m/seconds


In [3]:
'''Load data'''

'''Zone-related data'''

Taxi_Zones=np.load('./Data/NYC_Zones/Taxi_Zones.npy',allow_pickle=True).item()

Zone_list=np.load('./Data/NYC_Zones/Zone_list.npy',allow_pickle=True)

Zone_Center=np.load('./Data/NYC_Zones/Zone_Center.npy',allow_pickle=True).item()

Zone_Link=np.load('./Data/NYC_Zones/Zone_Link.npy',allow_pickle=True).item()

'''Link-related data'''

Link_list=np.load('./Data/NYC_Zones/Link_list.npy',allow_pickle=True)

Link_geometry=np.load('./Data/NYC_Zones/Link_geometry.npy',allow_pickle=True).item()


'''Point-related data'''

Points_list=np.load('./Data/NYC_Zones/Points_list.npy',allow_pickle=True)

Link_Point=np.load('./Data/NYC_Zones/Link_Point.npy',allow_pickle=True).item()

Point_coordinate=np.load('./Data/NYC_Zones/Point_coordinate.npy',allow_pickle=True).item()

Zone_Point=np.load('./Data/NYC_Zones/Zone_Point.npy',allow_pickle=True).item()

Point_zone=np.load('./Data/NYC_Zones/Point_zone.npy',allow_pickle=True).item()

'''Zone matrix'''

Connect_matrix=np.load('./Data/NYC_Zones/Connect_matrix.npy',allow_pickle=True)


'''Road network Object'''

G = nx.Graph()

G.add_nodes_from(Points_list)

G.add_weighted_edges_from(list(Link_Point.values()))

'''GeoSeries Object'''

polys = gp.GeoSeries(Taxi_Zones)




In [4]:
def Check_zones(pnt,polys):
    
    key='None'
    
    for k, geom in polys.items():
        
        if pnt.within(geom):
            
            key=k
            
            break
            
    return key

def Check_links(pnt,zone,Zone_Link,Link_middle):
    
    Dic={}
    
    for link in Zone_Link[zone]:
        
        middle_pnt=Link_middle[link]
        
        Dic[link]=pnt.distance(middle_pnt)
        
    return min(Dic, key=Dic.get)

def Check_points(pnt,Zone_Points,Point_coordinate):
    
    Dic={}
    
    for point in Zone_Points:
        
        coord=Point_coordinate[point]
        
        Dic[point]=pnt.distance(coord)
        
    return min(Dic, key=Dic.get)

def Get_neighbor_points(source,Point_coordinate,Points_list,radius):
    
    Point_candidates=list()
    
    for pnt in Points_list:
        
        dis=111000*(Point_coordinate[source].distance(Point_coordinate[pnt]))
    
        if dis<=radius:
            
            Point_candidates.append(pnt)
            
    return Point_candidates



def Random_walk(zone,Connect_matrix):
    
    zone_index=int(zone.split('_')[1])
    
    candidates=np.argwhere(Connect_matrix[zone_index]==1.0)
    
    dest=random.choice(candidates)[0]
    
    return 'Zone_'+str(dest)


def Get_path(G,source,target,Point_coordinate,Zone_Point):
    
    link_path=list()
    
    try:
    
        path=nx.shortest_path(G, source=source, target=target,weight='weight')

        shortest_dis=nx.shortest_path_length(G, source=source, target=target,weight='weight')
        
        link_path=path

        
    except:
        
        '''Path'''
        
        start_lng,start_lat=list(Point_coordinate[source].coords)[0][0],list(Point_coordinate[source].coords)[0][1]
        
        end_lng,end_lat=list(Point_coordinate[target].coords)[0][0],list(Point_coordinate[target].coords)[0][1]
        
        '''10 parts'''
        
        
        for i in range(1,10,1):
            
            pnt_lng=start_lng+(end_lng-start_lng)*(i/10)
            
            pnt_lat=start_lat+(end_lat-start_lat)*(i/10)
            
            pnt=Point(pnt_lng,pnt_lat)
            
            zone=Check_zones(pnt,polys)
            
            if zone != 'None':
            
                point=Check_points(pnt,Zone_Point[zone],Point_coordinate)
                
                if point not in [source,target] and point not in link_path:

                    link_path.append(point)
               
        link_path=[source]+link_path+[target]
        
        '''Distance'''
        
        shortest_dis=Point_coordinate[source].distance(Point_coordinate[target])*111000
    
        
    return link_path,shortest_dis

def reposition(point,zone,reposition_point,speed,Connect_matrix,Point_coordinate,G,Zone_Point):
    
    if reposition_point!=point:
        
        path=Get_path(G,point,reposition_point,Point_coordinate,Zone_Point)[0]
        
        dis=0
        
        if len(path)==2:
            
            point=reposition_point
            
        else:
        
            for i in range(1,len(path),1):

                dis+=Get_path(G,path[i-1],path[i],Point_coordinate,Zone_Point)[1]
                
                if dis>=speed*10:
                    
                    point=path[i]
                    
                    break
    else:
        
        '''update reposition destination'''
        
        reposition_candidates=Zone_Point[Random_walk(zone,Connect_matrix)]
        
        while len(reposition_candidates)==0:
            
            reposition_candidates=Zone_Point[Random_walk(zone,Connect_matrix)]
            
        
        reposition_point=random.choice(reposition_candidates)
        
    return [point,reposition_point]
        


In [5]:
Driver_df=pd.read_csv('./Data/NYC_Trips/Driver_df.csv')

Driver_df=Driver_df.drop(columns=['Unnamed: 0'])

Driver_df['Tuple']=Driver_df.apply(lambda x:reposition(x['Point'],x['Zone'],x['Reposition_Point'],speed,Connect_matrix,Point_coordinate,G,Zone_Point),axis=1)

Driver_df['Point']=Driver_df.apply(lambda x:x['Tuple'][0],axis=1)

Driver_df['Zone']=Driver_df.apply(lambda x:Point_zone[x['Point']],axis=1)

Driver_df['Reposition_Point']=Driver_df.apply(lambda x:x['Tuple'][1],axis=1)

Driver_df=Driver_df[['Driver_id','Order_id','Step','Point','Zone','Reposition_Point']]

Driver_df['Tuple']=Driver_df.apply(lambda x:reposition(x['Point'],x['Zone'],x['Reposition_Point'],speed,Connect_matrix,Point_coordinate,G,Zone_Point),axis=1)

Driver_df




,Driver_id,Order_id,Step,Point,Zone,Reposition_Point,Tuple
0,D0,Idle,2520,Point_3032,Zone_13,Point_842,"[Point_1169, Point_842]"
1,D1,Idle,2520,Point_2993,Zone_12,Point_1570,"[Point_5535, Point_1570]"
2,D2,Idle,2520,Point_1788,Zone_44,Point_5432,"[Point_1744, Point_5432]"
3,D3,Idle,2520,Point_792,Zone_59,Point_3032,"[Point_3318, Point_3032]"
4,D4,Idle,2520,Point_5285,Zone_47,Point_897,"[Point_1820, Point_897]"
5,D5,Idle,2520,Point_2518,Zone_6,Point_4303,"[Point_3843, Point_4303]"
6,D6,Idle,2520,Point_3419,Zone_47,Point_5066,"[Point_781, Point_5066]"
7,D7,Idle,2520,Point_5492,Zone_6,Point_806,"[Point_587, Point_806]"
8,D8,Idle,2520,Point_6030,Zone_12,Point_3521,"[Point_4626, Point_3521]"
9,D9,Idle,2520,Point_4243,Zone_12,Point_3858,"[Point_2167, Point_3858]"
